# <center> TP 3 : Apprentissage d'un mélange - Algorithme EM

## Chargement des données MNIST

In [1]:
%pylab
%matplotlib inline 

from mnist import load_mnist
import numpy as np
import visualize as vz

train_data, train_labels = load_mnist(dataset='training', path='./')
test_data, test_labels = load_mnist(dataset='testing', path='./')

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


Transform train_data and test_data into a column vector for each example.

In [2]:
train_data = np.reshape(train_data, (60000, 28 * 28)).T
test_data  = np.reshape(test_data,  (10000, 28 * 28)).T

## Partie 1 : Théorie

A partir de ce qui a été vu en cours, déduire les étapes de l'algorithme EM pour l'apprentissage des paramètres d'un mélange de loi de Bernoulli. Donner les formules de mise à jour des différents paramètres du modèle.

TODO

## Partie 2 : Implémentation

On vous demandera de tester différentes valeurs pour le nombre de composantes du mélange ainsi que différentes initialisations du modèle.

**1.** Implémenter cet algorithme et faire l'apprentissage sur les données binaires TRAINB obtenues précedemment. Attention à la manipulation des valeurs de probabilités (entre 0 et 1) qui deviennet rapidement nulles à cause de la précision des chiffres flottants).

In [3]:
# TODO

**2.** Afficher les moyennes des différentes distributions de Bernoulli ?

In [4]:
# TODO

**3.** En utilisant uniquement 10 composantes, est-il possible d'avoir un centre par classe ? Justifier.

In [5]:
# TODO

**4.** Tester un classifieur bayésien en utilisant un mélange pour chaque classe ? Quel est le taux de reconnaissance obtenu (tester 1, 2, 4, et 8 composantes par classe) ?

In [6]:
# TODO

## Partie 3 : Comparaison avec un GMM

Comparer les résultats obtenus avec le cas d'un mélange de gaussiennes sur les données brutes TRAIN et TEST.

**1.** Envisager le cas de gaussienne avec des matrices de covariance diagonale, ensuite tester la version avec matrices de covariance complètes.

In [7]:
# TODO

**2.** Attention à la dégénérescence de l'algorithme EM (matrice de covariance non inversible). Il faut voir comment gérer ce problème ainsi que l'initialisation.

In [8]:
# TODO

**3.** Proposer la meilleure solution pour avoir le plus haut taux de reconnaissance.

In [9]:
# TODO